In [1]:
# https://skymind.ai/wiki/generative-adversarial-network-gan
import keras
import numpy as np
import matplotlib.pyplot as plt

from keras.datasets import mnist
from keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import LeakyReLU
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers import Reshape
from keras.models import Model

In [2]:
class GAN():

    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Passing this fixed noise vector to generator while saving imagess
        self.fixed_noise = np.random.uniform(-1, 1, (25, self.latent_dim))

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        print(img.shape)
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()


        # Initial Dense layer to shape into a 8x8x256 tensor
        model.add(Dense(256 * 8 * 8, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Reshape((8, 8, 256)))

        # Upsampling layers
        model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))

        model.add(Conv2DTranspose(3, kernel_size=3, strides=1, padding='same', activation='tanh'))


        print("generator summary")
        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))

        print("discriminator summary")
        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()

        (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
        x_train = np.concatenate((x_train, x_test), axis=0)
        y_train = np.concatenate((y_train, y_test), axis=0)
        train_filter = (y_train == 1).flatten()
        X_train = x_train[train_filter]
        Y_train = y_train[train_filter]

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        #X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)


        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------
            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]


            # Sample noise as generator input
            noise = np.random.uniform(-1, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)
            print()

            # Train the discriminator
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))

            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)


            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.uniform(-1, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples
            # as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        #noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(self.fixed_noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("/content/UniformNoise_%d.png" % epoch)
        plt.close()


if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=6001, batch_size=64, save_interval=600) #epoch 30,000

Streaming output truncated to the last 5000 lines.
4335 [D loss: 0.658619, acc.: 62.50%] [G loss: 0.737618]
1/1 [==============================] - 0s 24ms/step

4336 [D loss: 0.687882, acc.: 48.44%] [G loss: 0.731250]
1/1 [==============================] - 0s 28ms/step

4337 [D loss: 0.668254, acc.: 54.69%] [G loss: 0.718380]
1/1 [==============================] - 0s 28ms/step

4338 [D loss: 0.683488, acc.: 54.69%] [G loss: 0.717346]
1/1 [==============================] - 0s 28ms/step

4339 [D loss: 0.660340, acc.: 54.69%] [G loss: 0.712412]
1/1 [==============================] - 0s 17ms/step

4340 [D loss: 0.701295, acc.: 35.94%] [G loss: 0.703195]
1/1 [==============================] - 0s 18ms/step

4341 [D loss: 0.701437, acc.: 39.06%] [G loss: 0.722853]
1/1 [==============================] - 0s 22ms/step

4342 [D loss: 0.656756, acc.: 54.69%] [G loss: 0.732195]
1/1 [==============================] - 0s 17ms/step

4343 [D loss: 0.721434, acc.: 48.44%] [G loss: 0.737350]
1/1 [=======